In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here are several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pet-adoption/Dataset/train.csv
/kaggle/input/pet-adoption/Dataset/test.csv


In [2]:
train=pd.read_csv("../input/pet-adoption/Dataset/train.csv")
test=pd.read_csv("../input/pet-adoption/Dataset/test.csv")
train.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [3]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [4]:
train.shape,test.shape

((18834, 11), (8072, 9))

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8072 entries, 0 to 8071
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pet_id        8072 non-null   object 
 1   issue_date    8072 non-null   object 
 2   listing_date  8072 non-null   object 
 3   condition     7453 non-null   float64
 4   color_type    8072 non-null   object 
 5   length(m)     8072 non-null   float64
 6   height(cm)    8072 non-null   float64
 7   X1            8072 non-null   int64  
 8   X2            8072 non-null   int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 567.7+ KB


In [7]:
train['breed_category'].value_counts()

0.0    9000
1.0    8357
2.0    1477
Name: breed_category, dtype: int64

In [8]:
#cheak
a=train['breed_category'][(np.isnan(train['condition']))]
a.value_counts()

2.0    1477
Name: breed_category, dtype: int64

It means all the missing values belong to a single particular label. So, we can fill them with a unique value like -1.

**combine train and test data** 

In [9]:
#copy all test id to create submission file
test_id=test['pet_id']
#save the train left...it will use when the combine data will split into the previous train and test data after doing feature engineering
ntrain=train.shape[0]

In [10]:
#save target variable i.e label
y1=train['breed_category']
y2=train['pet_category']

In [11]:
#combine test and train data
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['breed_category','pet_category'], axis=1, inplace=True)

now it's ready to do feature engineering

In [12]:
all_data['condition'].value_counts()

1.0    9747
0.0    8966
2.0    6097
Name: condition, dtype: int64

In [13]:
all_data['condition'].fillna(-1,inplace=True)

In [14]:
all_data['condition'].value_counts()

 1.0    9747
 0.0    8966
 2.0    6097
-1.0    2096
Name: condition, dtype: int64

In [15]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26906 entries, 0 to 26905
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pet_id        26906 non-null  object 
 1   issue_date    26906 non-null  object 
 2   listing_date  26906 non-null  object 
 3   condition     26906 non-null  float64
 4   color_type    26906 non-null  object 
 5   length(m)     26906 non-null  float64
 6   height(cm)    26906 non-null  float64
 7   X1            26906 non-null  int64  
 8   X2            26906 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 1.8+ MB


So, there is no null values in any column of all_data.

**seasonal feature(new feature)**

In [16]:
all_data['issue_date']=pd.to_datetime(all_data['issue_date'])
all_data['listing_date']=pd.to_datetime(all_data['listing_date'])


In [17]:
x=[]
for d in all_data['issue_date']:
    y=d.month
    x.append(y)
all_data['issue_month']=x

In [18]:
x=[]
for d in all_data['listing_date']:
    y=d.month
    x.append(y)
all_data['listing_month']=x

In [19]:
x=[]
for d in all_data['listing_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)
all_data['modified_listing_date']=x

In [20]:
x=[]
for d in all_data['issue_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)
all_data['modified_issue_date']=x

In [21]:
all_data['took_time']=abs(all_data['modified_listing_date']-all_data['modified_issue_date'])

**Modified pet id and extract important feature**

* when the dataset was creating, it might be happened that it was listing with special id for a particular animal.
* just like ANSL_69903.so,all ANSL_6**** i.e., the id starts with 6 maybe a particular animal because we can see that all the ids are not coming serially.
* so extracting the 1st and both 1st and 2nd numbers maybe a good feature.

In [22]:

all_data['1stnum'] = all_data['pet_id'].str[:6]
all_data['1st2num'] = all_data['pet_id'].str[:7]

split back to the train and test data

In [23]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [24]:
#drop some unnecessary features
x=train.drop(['pet_id','issue_date','listing_date','modified_issue_date'],axis=1)
test=test.drop(['pet_id','issue_date','listing_date','modified_issue_date'],axis=1)


* **handle categorical variable**

In [25]:
x.select_dtypes(exclude='number').columns.to_list()

['color_type', '1stnum', '1st2num']

One Hot endcode

In [26]:
x.shape

(18834, 12)

In [27]:
x=pd.get_dummies(x)
test=pd.get_dummies(test)


In [28]:
x.shape,test.shape

((18834, 97), (8072, 95))

In [29]:
a=set(x.columns)-set(test.columns)

In [30]:
a=list(a)
a

['color_type_Black Tiger', 'color_type_Brown Tiger']

In [31]:
x=x.drop(a,axis=1)

In [32]:
x.shape,test.shape

((18834, 95), (8072, 95))

In [33]:
#again combining
all_data = pd.concat((x, test)).reset_index(drop=True)

**Scaling using StandardScaler()**

In [34]:
from sklearn import preprocessing
# Get column names first
names = all_data.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(all_data)
all_data = pd.DataFrame(scaled_df, columns=names)

In [35]:
x = all_data[:ntrain]
test = all_data[ntrain:]

In [36]:
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test=train_test_split(x,y2,test_size=0.2,random_state=44,shuffle=True)

In [37]:
from xgboost import XGBClassifier

In [38]:
model1 = XGBClassifier()
model1.fit(x1_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
#new_feat is new feature i.e the predicted pet_category of model 1 for train data
new_feat=model1.predict(x)
#output1 is new first output i.e the predicted pet_category of model 1 for test data
output1=model1.predict(test)
#vld1 is validation 1 i.e we'll check score with the predicted result of validation data of model 1
vld1=model1.predict(x1_test)

In [40]:
x2 = pd.DataFrame(x, columns=names)
test2 = pd.DataFrame(test, columns=names)

In [41]:
#the predicted pet_category of model 1 for train data is used as a input variable or feature of the train data of model 2
x2['output1']=new_feat
#the predicted pet_category of model 1 for test data is used as a input variable or feature of the test data of model 2
test2['output1']=output1

**split data for model 2**

In [42]:
x2_train,x2_test,y2_train,y2_test=train_test_split(x2,y1,test_size=0.2,random_state=44)

In [43]:
model2 = XGBClassifier()
model2.fit(x2_train, y2_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [44]:
#output 2 is the predicted breed_category of model 2 for test data
output2=model2.predict(test)
#vld2 is validation 2 i.e we'll check score with the predicted result of validation data of model 2
vld2=model2.predict(x2_test)

In [45]:
from sklearn.metrics  import f1_score

In [46]:
s1=f1_score(y1_test,vld1,average='weighted')
s2=f1_score(y2_test,vld2,average='weighted')
accuracy=100*((s1+s2)/2)
accuracy

90.43734390069241

In [47]:
sub_new=pd.DataFrame({
    "pet_id":test_id,
    "breed_category":output2,
    "pet_category":output1
})
sub_new.to_csv("petadopt.csv",index=False)